In [42]:
import numpy as np
import pickle
import time
from sklearn.cross_validation import train_test_split

In [4]:
#load data
id_num = pickle.load(open('id_num.txt', 'rb'))
train_X = pickle.load(open('train_data.txt', 'rb'))
train_y = pickle.load(open('train_y.txt', 'rb'))
non_null = pickle.load(open('non_null.txt', 'rb'))

In [5]:
types = []
for i in range(non_null.shape[1]):
    data = train_X[:, i]
    real = non_null[:, i]
    rd = data[real]
    x = str(type(rd[0]))
    if 'str' in x:
        types.append('str')
    elif 'float' in x:
        types.append('float')
    else:
        types.append('int')

In [8]:
# fix null in string column first, use the most frequency one

In [30]:
def dict2list(dic):
    keys = dic.keys()
    vals = dic.values()
    lst = [(key, val) for key, val in zip(keys, vals)]
    return lst

# input one array and return the most frequent one
def MostFrequentOne(column):
    itemfreq = {}
    for i in column:
        if i not in itemfreq:
            itemfreq[i] = 1
        else:
            itemfreq[i] += 1
    tmp_dict = sorted(dict2list(itemfreq), key=lambda d:d[1], reverse=True)
    return tmp_dict[0]

# return a reverse boolean column
def BoolReverse(column):
    x = np.zeros(len(column))
    x[column] = 1
    y = (x==0)
    return y

#If A column with only one value equals to zero, then it shall be excluded
def OneValueIfNeo(column):
    x = np.sum(column==0)
    if x > 0:
        return True
    else:
        return False

# normalize raw, no mean and std, they shall be computed
def Normalize_Raw(column):
    mean = np.mean(column)
    std = np.std(column)
    return mean, std
# normalize one column by mean and std
def Norm(column, mean, std):
    return (column - mean)/std

# 0-1 average
def NeoOneMean_Raw(column):
    maxv = np.max(column)
    minv = np.min(column)
    return maxv, minv
# mean one column to 0-1
def NeoOneMean(column, maxv, minv):
    return (column - minv) / (maxv - minv)

In [34]:
# fix the non or null
def OneStrColumnFix(column, nn):
    mfo = MostFrequentOne(column[nn])
    rn = BoolReverse(nn)
    column[rn] = mfo
    return column

# one_hot to map str to int
def OneHot_Str(column, retdict = 0):
    syms = set(column)
    sim = {}
    count = 0
    for s in syms:
        sim[s] = count
        count += 1
    cl = len(column)
    retoh = np.zeros((cl, count))
    for i in range(cl):
        s = column[i]
        mi = sim[s]
        retoh[i, mi] = 1.0
    if retdict == 0:
        return retoh
    else:
        return retoh, sim

In [32]:
#fix the null in str and one_hot it, concatenate the one_hot columns to the end
for i in range(len(types)):
    t = types[i]
    if t == 'str':
        column = train_X[:, i]
        nn = non_null[:, i]
        column_fixed = OneStrColumnFix(column, nn)
        train_X[:, i] = column_fixed
        one_hots = OneHot_Str(column_fixed)
        train_X = np.concatenate((train_X, one_hots), axis=1)
        print(i)

0
231
749
771
942
2379
3831
4106
5915
6512


In [36]:
# now fix int columns and float columns
methods = []
#method = 2, normalization 3, meanNeoOne
method = 2
for i in range(len(types)):
    t = types[i]
    if t == 'str':
        methods.append((i, 0))
    else:
        data = train_X[:, i]
        real = non_null[:, i]
        numv = len(set(data[real]))
        #fix null
        if np.sum(real) < 500:
            rn = BoolReverse(real)
            if numv == 1:
                data[rn] = data[real][0]
            else:
                data[rn] = np.mean(data[real])
        #fix null, then manipulate the column
        if numv == 1:
            data = np.ones(train_X.shape[0])
            methods.append((i, 1))
        else:
            if method == 2:
                mean, std = Normalize_Raw(data)
                data = (data-mean)/std
                methods.append((i, mean, std))
            elif method == 3:
                maxv, minv = NeoOneMean_Raw(data)
                data = (data - minv)/(maxv - minv)
                methods.append((i, maxv, minv))
        

In [37]:
def MSE(y_pred, y):
    return np.mean(np.square(y-y_pred))

In [39]:
# now delete all str column
stridxs = []
for i in range(len(types)):
    t = types[i]
    if t == 'str':
        stridxs.append(i)
train_X = np.delete(train_X, stridxs, axis=1)
print(train_X.shape)

(500, 7989)


In [60]:
y_m = np.mean(train_y)
y_s = np.std(train_y)
train_y = (train_y-y_m)/y_s

In [63]:
from sklearn.decomposition import PCA

In [110]:
cn = 15 
pca = PCA(n_components=cn) 
pca.fit(train_X) 
psum = 0 
pe = pca.explained_variance_ratio_
for i in range(cn): 
    e = pe[i] 
    psum += e 
    print(i, psum) 
X_new = pca.transform(train_X) 
print("new shape", X_new.shape)

0 0.999999968443
1 0.999999983275
2 0.999999992392
3 0.999999999294
4 0.999999999902
5 0.99999999996
6 0.999999999979
7 0.999999999994
8 0.999999999997
9 0.999999999998
10 0.999999999999
11 0.999999999999
12 0.999999999999
13 1.0
14 1.0
new shape (500, 15)


In [111]:
for i in range(cn):
    data = X_new[:, i]
    mean, std = Normalize_Raw(data)
    data = (data - mean) / std
    X_new[:, i] = data 

In [143]:
print(X_new)

[[ 1.06191318  0.30225538 -0.32706471 ..., -1.63407943 -0.46975306
   0.38670017]
 [-0.94169659 -0.15128664 -1.00714045 ..., -0.52040649  0.6231931
   1.90539893]
 [-0.9416966  -0.37398339 -0.82482787 ..., -0.51219074  0.04798749
   0.62511736]
 ..., 
 [-0.9416966  -1.16272784  0.73796304 ...,  1.01348782 -0.94394529
  -1.9971444 ]
 [ 1.06191317 -1.03518188  1.78971803 ...,  1.39034844  0.23757798
  -0.19830898]
 [-0.9416966  -0.30308045 -1.04353053 ...,  0.59632295 -1.28882662
  -2.79696366]]


# Now model and prediction

In [145]:
np.random.seed(int(time.time()))
idxs = np.arange(len(train_X))
np.random.shuffle(idxs)
train_X = train_X[idxs]
train_y = train_y[idxs]
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.2, random_state=33)

In [165]:
pca = PCA(n_components=15) 
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)
for i in range(cn):
    mean, std = np.mean(X_train[:, i]), np.std(X_train[:, i])
    X_train[:, i] = (X_train[:, i] - mean) / std
    X_test[:, i] = (X_test[:, i] - mean) / std

In [166]:
X_test.shape

(100, 15)

In [45]:
import tensorflow as tf

In [169]:
column_len = X_train.shape[1]

In [170]:
def model(input_data):
    l1 = tf.layers.dense(input_data, 1000)
    l2 = tf.layers.dense(l1, 100)
    l3 = tf.layers.dense(l2, 10)
    l4 = tf.layers.dense(l3, 1)
    return l4

In [171]:
def loss_f(y, y_pred):
    y = y*y_s+y_m
    y_pred = y_pred*y_s+y_m
    return tf.reduce_mean(tf.square(y-y_pred))

In [172]:
input_X = tf.placeholder(shape=[None, column_len], dtype=tf.float32)
input_y = tf.placeholder(shape=[None], dtype=tf.float32)

In [173]:
pred_y = model(input_X)
loss = loss_f(pred_y, input_y) 
opt = tf.train.AdamOptimizer(0.001).minimize(loss)

In [174]:
sess = tf.Session()
tf.global_variables_initializer().run(session=sess)

In [175]:
for i in range(101):
    _, l = sess.run([opt, loss], feed_dict={input_X:X_train, input_y:y_train})
    #l = sess.run(loss, feed_dict={input_X:X_test, input_y:y_test})
    print(i, l)

0 0.0461244
1 0.0427383
2 0.045103
3 0.0420402
4 0.0403142
5 0.0413477
6 0.0423951
7 0.0419504
8 0.0408584
9 0.040265
10 0.0404792
11 0.0409928
12 0.0412035
13 0.0409727
14 0.0405532
15 0.040262
16 0.0402563
17 0.0404607
18 0.0406522
19 0.0406595
20 0.040498
21 0.040311
22 0.0402206
23 0.0402491
24 0.0403385
25 0.0404097
26 0.0404115
27 0.0403459
28 0.0402606
29 0.0402115
30 0.0402209
31 0.0402667
32 0.0403042
33 0.0403049
34 0.0402729
35 0.0402332
36 0.0402103
37 0.0402131
38 0.0402332
39 0.0402526
40 0.0402548
41 0.0402387
42 0.0402177
43 0.0402068
44 0.0402101
45 0.040221
46 0.0402292
47 0.0402287
48 0.0402201
49 0.0402103
50 0.040206
51 0.0402086
52 0.0402144
53 0.040218
54 0.0402163
55 0.0402112
56 0.0402067
57 0.0402061
58 0.0402084
59 0.0402112
60 0.0402117
61 0.04021
62 0.0402072
63 0.0402055
64 0.0402063
65 0.0402078
66 0.0402088
67 0.0402082
68 0.0402067
69 0.0402058
70 0.0402058
71 0.0402065
72 0.0402071
73 0.040207
74 0.0402064
75 0.0402059
76 0.0402057
77 0.0402061
78 0.04

In [177]:
import pandas as pd

In [198]:
test_data = pd.read_excel('测试A.xlsx')

In [192]:
testlb = pd.read_csv('测试A-答案模板.csv', header=None)

In [196]:
import pickle
firstdelete = pickle.load(open('firstdelete.txt', 'rb'))

In [199]:
testnn = test_data.isnull().values
test_data = test_data.values
test_data = np.delete(test_data, firstdelete, axis=1)
testnn = np.delete(testnn, firstdelete, axis=1)
test_nn = testnn[:, 1:]
test_id = test_data[:, 0]
test_X = test_data[:, 1:]

In [207]:
tmpn = np.zeros(test_nn.shape)
tmpn[test_nn] = 1
test_nn = tmpn==0

In [209]:
ttypes = []
for i in range(test_X.shape[1]):
    nn = test_nn[:, i]
    data = test_X[:, i]
    rd = data[nn]
    strtype = str(type(rd[0]))
    if 'str' in strtype:
        ttypes.append('str')
    elif 'float' in strtype:
        ttypes.append('float')
    else:
        ttypes.append('int')

In [211]:
t1 = np.array(types)
t2 = np.array(ttypes)

In [213]:
print(np.sum(t1==t2), len(t1))

7674 7965
